# CoreML


Core ML is an Apple framework that allows developers to integrate machine learning/deep learning models into their applications. However, it does not support model creation and training, i.e., you first need to create the model in a framework like TensorFlow or PyTorch, then you can convert and use it. There are two ways you can convert your machine learning model from the framework of your choice to the Core ML model format: through an intermediary model format like ONNX or by using Apple’s own CoreMLTools Python library. 

To read about it more, please refer [this](https://analyticsindiamag.com/converting-pytorch-and-tensorflow-models-into-apple-core-ml-using-coremltools/) article.

# Code Implementation

### Install Core ML Tools 

In [ ]:
!python -m pip install pip --upgrade --user -q
!python -m pip install numpy pandas seaborn matplotlib scipy sklearn statsmodels tensorflow keras --user -q

In [ ]:
!python -m pip install --upgrade coremltools --user -q

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## From TensorFLow to Core ML Model

In [ ]:
import tensorflow as tf
import coremltools as ct

model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ]
)


(X_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# fit the model
model.fit(X_train, y_train, batch_size=32)

In [ ]:
type(model)

In [ ]:
mlmodel = ct.convert(model)

In [ ]:
type(mlmodel)

### Only works on macOS version 10.13 or later

In [ ]:
# !wget -O 2.jpg https://cdn2.vectorstock.com/i/1000x1000/95/51/floral-numbers-colorful-flowers-number-2-vector-14029551.jpg

In [ ]:
 import numpy as np
 from IPython.display import display
 from PIL import Image
 img = Image.open('2.jpg').resize((28, 28)).convert('L')
 display(img)
 output =  mlmodel.predict({"input_1": img})
 print(output)

### From PyTocrch to Core ML Model

In [ ]:
import torch
import torchvision
import coremltools as ct

# Load a pre-trained version of MobileNetV2
torch_model = torchvision.models.mobilenet_v2(pretrained=True)

In [ ]:
# Set the model in evaluation mode
torch_model.eval()

In [ ]:
# Trace with random data
random_input = torch.rand(1, 3, 224, 224) 
traced_model = torch.jit.trace(torch_model, random_input)

In [ ]:
# Download class labels (from a separate file)
import urllib
label_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
class_labels = urllib.request.urlopen(label_url).read().decode("utf-8").splitlines()

class_labels = class_labels[1:] # remove the first class which is background
assert len(class_labels) == 1000

In [ ]:
# Convert to Core ML using the Unified Conversion API
model = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="input_1", shape=random_input.shape)]
)

In [ ]:
# Save model
# model.save("MobileNetV2.mlmodel")

### Only works on macOS version 10.13 or later

In [ ]:
# !wget -O daisy.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Daisy_G%C3%A4nsebl%C3%BCmchen_Bellis_perennis_01.jpg/1280px-Daisy_G%C3%A4nsebl%C3%BCmchen_Bellis_perennis_01.jpg

In [ ]:
from PIL import Image
example_image = Image.open("daisy.jpg").resize((224, 224))

# Make a prediction using Core ML
out_dict = model.predict({"input_1": example_image})

# Print out top-1 prediction
print(out_dict["classLabel"])